In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)
include("9msctrl.jl")
include("Initial condition variation.jl")
include("Perturbation time.jl")
import LinearAlgebra: norm
using PyCall
plt = pyimport("matplotlib.pyplot");

┌ Info: Recompiling stale cache file C:\Users\User\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


In [2]:
timestepdifference = 0.5
T = 10000

objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
function objfun(t, u, dudt, I, dIdt)
    return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
end
function objfun(xq::Coupled)
    return objfun(xq[1])
end

function PlotobjRe(u0::AbstractVector,Re::AbstractVector)
    output = zeros(length(Re))
    for i in 1:length(Re)
        u0_local = copy(u0)
        I0 = Float64[0.0]
        Reynolds = Re[i]
        
        f  = NineModeSystem(Reynolds)
        ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(timestepdifference))
        mon = Monitor(couple(zeros(9), zeros(1)), objfun)
        ϕ(couple(u0_local, I0), (0, T), reset!(mon))
        
        output[i] = I0[1]/T
    end
#     PyPlot.plot(Re,output)
#     xlabel(L"Re")
#     ylabel(L"J(t)")
    return output
end

PlotobjRe (generic function with 1 method)

In [3]:
num = 50
Re_range = 10:10:900
Real_output = []
fig,ax = plt.subplots()
for i in 1:num
    u0 = rand(9)
    y_output = PlotobjRe(u0, Re_range)
    Real_output = push!(Real_output, y_output)
    ax.plot(Re_range, y_output, color = "grey", linestyle="dashed",linewidth = 0.05)
end
Averaged_output = sum(Real_output)./length(Real_output)
ax.plot(Re_range, Averaged_output, color = "black", linewidth = 0.5)

1-element Array{PyObject,1}:
 PyObject <matplotlib.lines.Line2D object at 0x0000000047A1FD08>

In [4]:
xlabel(L"Re")
ylabel(L"J(Re)")

PyObject Text(42.722222222222214, 0.5, '$J(Re)$')